# Data wrangling‬ for 311 Service requests

## Objective:

   Data wrangling and visualization tasks will be performed to ensure that raw data is transformed into a‬ clean format and then visualized effectively to develop valuable insights. Data wrangling‬ involves handling null values by imputing mean or median values in missing values of columns longitude‬ or latitude‬. Duplicate records are identified and removed, standardizing data types to‬ ensure consistency, such as converting Floating Timestamp to datetime for the date columns‬. Data transformations like encoding categorical variables (e.g., encoding status description “open”=> 0 and “closed” => 1) and normalizing numerical data for easier analysis. Feature engineering to derive‬ new columns, such as calculating response time (e.g., the difference between closed date and‬ requested date will be the response time), and additionally, grouping and mapping subdivisions into‬ broader categories, such as combining detailed service name into high-level service groups for providing‬ clearer insights‬. When unusual values are found in numerical columns, these are outliers,‬ which can be treated by capping or removal to prevent skewed results‬.
    

In [72]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

import re
import pandas as pd
from tabulate import tabulate


### Part 1: Data Cleaning and Preprocessing

### 1.1 Load and Inspect the Dataset

• Load the dataset and display its shape, column names, and data types.

• Identify and list the number of missing values in each column.

In [74]:
# load data
df = pd.read_csv('/Users/anithajoseph/Documents/UofC/DATA601/CSVFiles/311_Service_Requests_2yrs.csv')
print("----------------------------------------------------------------------------")
print("\033[1m"+"Data Analysis and Visualization of Building Emergency Benchmarking"+"\033[0m")
print("----------------------------------------------------------------------------")

#display shape, columns, and data types
print("1.\tShape of the Dataset:", df.shape)
print("2.\tNumber of records or rows of the DataFrame:", df.shape[0])
print("3.\tColumns and Data types of each column:\n", df.dtypes)

# Inspect data
missingDataSum = df.isna().sum()
missingDataPercentage = (df.isnull().mean() * 100).round(2)
missingData = pd.DataFrame({
    "Missing Count": missingDataSum,
    "Missing Percentage": missingDataPercentage
})

pd.options.display.float_format = '{:.2f}'.format
print("\n\033[1m"+"Missing Count per column:"+"\033[0m")
print(tabulate(missingData, headers='keys', tablefmt='fancy_grid'))

#The dataframe(DF) is copied to another DF variable if in case there is a need for original DF
originalDF =df

----------------------------------------------------------------------------
Data Analysis and Visualization of Building Emergency Benchmarking
----------------------------------------------------------------------------
1.	Shape of the Dataset: (1093918, 15)
2.	Number of records or rows of the DataFrame: 1093918
3.	Columns and Data types of each column:
 service_request_id     object
requested_date         object
updated_date           object
closed_date            object
status_description     object
source                 object
service_name           object
agency_responsible     object
address               float64
comm_code              object
comm_name              object
location_type          object
longitude             float64
latitude              float64
point                  object
dtype: object

Missing Count per column:
╒════════════════════╤═════════════════╤══════════════════════╕
│                    │   Missing Count │   Missing Percentage │
╞════════════════════╪═

### 1.2 Handling Missing Data

• Drop columns with more than 10% missing values.

In [76]:
#Drop columns with missing percentage >40%
columnNameDropped = missingDataPercentage[missingDataPercentage >= 40].index.tolist()
print("\nColumns with missing percentage more than 40% missing values are:\n", columnNameDropped)

df = df.drop(columns = missingDataPercentage[missingDataPercentage > 40].index)


Columns with missing percentage more than 40% missing values are:
 ['address']


### 1.3 Date and Time Handling:

•	Convert the Date column to a datetime object.

•	Create new columns for the year, month, and day of the week for requested, updated and closed date columns.

•	Add a column indicating whether each date falls on a weekend.

•	Add a column for time duration to calculate the time took to close the request.

In [78]:
df['requested_date'] = pd.to_datetime(df['requested_date'], format = '%Y/%m/%d %I:%M:%S %p')
print(f"Data type of 'requested_date': {df['requested_date'].dtype}")
df['request_year'] = df['requested_date'].dt.year
df['request_month'] = df['requested_date'].dt.month
df['request_day'] = df['requested_date'].dt.day
#df['requested_date'] = df['requested_date'].astype('datetime64[ns]')
print(df['requested_date'].dtype)


df['updated_date'] = pd.to_datetime(df['updated_date'], format = '%Y/%m/%d %I:%M:%S %p')
# Extract year, month, and day directly from the 'timestamp' column
df['update_year'] = df['updated_date'].dt.year
df['update_month'] = df['updated_date'].dt.month
df['update_day'] = df['updated_date'].dt.day
#display(df.head(5))


df['closed_date'] = pd.to_datetime(df['closed_date'], format = '%Y/%m/%d %I:%M:%S %p')
print(df['closed_date'].dtype)
df['closed_date'] = df['closed_date'].fillna(pd.NaT)
df['closed_year'] = df['closed_date'].dt.year
df['closed_month'] = df['closed_date'].dt.month
df['closed_day'] = df['closed_date'].dt.day
df.loc[df['closed_date'].isna(), ['closed_year', 'closed_month', 'closed_day']] = 0
df[['closed_year', 'closed_month', 'closed_day']] = df[['closed_year', 'closed_month', 'closed_day']].astype('Int32')

#display(df.head(5))
df['request_day'] = pd.to_datetime(df['request_day'], errors='coerce')
print(df['request_day'].isnull().sum())
df['is_weekend'] = df['request_day'].dt.weekday >= 5

df['time_closing'] = df['closed_date'] - df['requested_date']
display(df.head())

Data type of 'requested_date': datetime64[ns]
datetime64[ns]
datetime64[ns]
0


,service_request_id,requested_date,updated_date,closed_date,status_description,source,service_name,agency_responsible,comm_code,comm_name,...,request_month,request_day,update_year,update_month,update_day,closed_year,closed_month,closed_day,is_weekend,time_closing
0,23-00000797,2023-01-02,2023-01-10,2023-01-10,Closed,Other,Finance - ONLINE TIPP Agreement Request,CFOD - Finance,NaN,NaN,...,1,1970-01-01 00:00:00.000000002,2023,1,10,2023,1,10,False,8 days
1,23-00001045,2023-01-02,2024-01-11,2024-01-11,Closed,Other,Active Living Program Application,CS - Recreation and Social Programs,NaN,NaN,...,1,1970-01-01 00:00:00.000000002,2024,1,11,2024,1,11,False,374 days
2,23-00001163,2023-01-02,2023-01-06,2023-01-06,Closed,Phone,CN - Registered Social Worker Letter,CS - Calgary Neighbourhoods,NaN,NaN,...,1,1970-01-01 00:00:00.000000002,2023,1,6,2023,1,6,False,4 days
3,23-00001191,2023-01-02,2024-05-19,2023-01-10,Closed,Other,CT - Lost Property,OS - Calgary Transit,NaN,NaN,...,1,1970-01-01 00:00:00.000000002,2024,5,19,2023,1,10,False,8 days
4,23-00001584,2023-01-02,2023-01-04,2023-01-04,Closed,Other,Recreation - Arena Booking Application,CS - Calgary Recreation,NaN,NaN,...,1,1970-01-01 00:00:00.000000002,2023,1,4,2023,1,4,False,2 days


### 1.4 Create additional columns:

•	Add a column to see if the request is duplicate or not(Yes means duplicate and No means not a duplicate request).

In [80]:
df['duplicate_request'] = df['status_description'].str.contains(r'Duplicate \(Closed\)', regex=True)

# Convert the boolean values to 'yes'/'no'
df['duplicate_request'] = df['duplicate_request'].replace({True: 'Yes', False: 'No'})

# Checking if converted to yes or no
df_subset = df.iloc[150:166]  # Python slicing includes 150 but excludes 166
display(df_subset)

,service_request_id,requested_date,updated_date,closed_date,status_description,source,service_name,agency_responsible,comm_code,comm_name,...,request_day,update_year,update_month,update_day,closed_year,closed_month,closed_day,is_weekend,time_closing,duplicate_request
150,23-00000917,2023-01-02,2023-01-04,2023-01-04,Closed,Other,CT - Lost Property,TRAN - Calgary Transit,NaN,NaN,...,1970-01-01 00:00:00.000000002,2023,1,4,2023,1,4,False,2 days,No
151,23-00000924,2023-01-02,2024-05-18,2023-01-05,Duplicate (Closed),Other,Roads - Snow and Ice Control,OS - Mobility,MCT,MCKENZIE TOWNE,...,1970-01-01 00:00:00.000000002,2024,5,18,2023,1,5,False,3 days,Yes
152,23-00000925,2023-01-02,2023-01-02,2023-01-02,Closed,App,Roads - Streetlight Maintenance,TRAN - Roads,ROY,ROYAL OAK,...,1970-01-01 00:00:00.000000002,2023,1,2,2023,1,2,False,0 days,No
153,23-00000926,2023-01-02,2023-01-05,2023-01-05,Closed,Other,CBS - Planning and Development - After Hours SR,PD - Calgary Building Services,MID,MIDNAPORE,...,1970-01-01 00:00:00.000000002,2023,1,5,2023,1,5,False,3 days,No
154,23-00000942,2023-01-02,2023-01-03,2023-01-03,Closed,Phone,CBS Inspection - Residential Improvement Proje...,PD - Calgary Building Services,TAR,TARADALE,...,1970-01-01 00:00:00.000000002,2023,1,3,2023,1,3,False,1 days,No
155,23-00000931,2023-01-02,2023-01-04,2023-01-04,Closed,Other,WRS - Cart Management,UEP - Waste and Recycling Services,VAR,VARSITY,...,1970-01-01 00:00:00.000000002,2023,1,4,2023,1,4,False,2 days,No
156,23-00000943,2023-01-02,2023-01-16,2023-01-02,Closed,Phone,WATS - Sewage Back-up,UEP - Water Services,CED,CEDARBRAE,...,1970-01-01 00:00:00.000000002,2023,1,16,2023,1,2,False,0 days,No
157,23-00000930,2023-01-02,2023-01-02,2023-01-02,Closed,Other,CT AC - Trip Feedback - Checker Taxi,Tranc - Calgary Transit,NaN,NaN,...,1970-01-01 00:00:00.000000002,2023,1,2,2023,1,2,False,0 days,No
158,23-00000927,2023-01-02,2023-01-17,2023-01-17,Closed,Other,CT - Bus Stops,TRAN - Calgary Transit,BRI,BRIDLEWOOD,...,1970-01-01 00:00:00.000000002,2023,1,17,2023,1,17,False,15 days,No
159,23-00000938,2023-01-02,2023-01-30,2023-01-30,Closed,Other,WATS - Water Meter Issues,UEP - Water Services,BRI,BRIDLEWOOD,...,1970-01-01 00:00:00.000000002,2023,1,30,2023,1,30,False,28 days,No


## Service name analysis 

In [82]:
service_nunique_values=df['service_name'].nunique()
print("Count of distinct of all service names:", service_nunique_values)

service_unique_values=df['service_name'].unique()
display(service_unique_values)

service_unique_values_cnt=df['service_name'].value_counts()
print(service_unique_values_cnt)

df['new_serviceNames'] = df['service_name'].str.split(' -').str[0] #extracted main service name and removed sub division of that.
sn_unique_values = df['new_serviceNames'].unique()
print(sn_unique_values)

unique_values = df['new_serviceNames'].nunique()
print("Number of Unique values: ",unique_values)
unique_values_cnt = df['new_serviceNames'].value_counts()
print(unique_values_cnt)

#count of particular value
count = (df['new_serviceNames'] == 'AS').sum()
count

Count of distinct of all service names: 640


array(['Finance - ONLINE TIPP Agreement Request',
       'Active Living Program Application',
       'CN - Registered Social Worker Letter', 'CT - Lost Property',
       'Recreation - Arena Booking Application',
       'CT AC - Trip Feedback - CTA',
       'REC - Southland Leisure Centre Inquiry',
       'Parks - Snow and Ice Concerns - WAM', 'AS - Pick Up Stray',
       'AS - No Cost Spay or Neuter Program Inquiries',
       'WATS - Water Quality', 'Bylaw - Snow and Ice on Sidewalk',
       'Parks - Parks and Open Spaces Bookings',
       'Roads - Snow and Ice Control', 'Corporate - Graffiti Concerns',
       'Finance - Property Tax Account Inquiry',
       'CBS Inspection - Commercial or Multi-Family', '311 Contact Us',
       'WATS - Water Pressure Issues',
       'WATR - Erosion and Sediment Control', 'CT - Bus Stops',
       'Roads - Traffic Signal Timing Inquiry', 'CBS Inspection - Gas',
       'WRS - Cart Management', 'WATS - Sewer Maintenance',
       'After Hours Transit - Gen

service_name
WRS - Cart Management                      51648
Finance - Property Tax Account Inquiry     37459
Bylaw - Snow and Ice on Sidewalk           34438
Finance - ONLINE TIPP Agreement Request    28318
Corporate - Graffiti Concerns              22096
                                           ...  
CAI - Employee Complaint - Compliment          1
WATR - Water Brochure                          1
PSD - Major Mobility - Paving Program          1
WRS - Chatbot Feedback                         1
CPI - Employee Complaint - Compliment          1
Name: count, Length: 640, dtype: int64
['Finance' 'Active Living Program Application' 'CN' 'CT' 'Recreation'
 'CT AC' 'REC' 'Parks' 'AS' 'WATS' 'Bylaw' 'Roads' 'Corporate'
 'CBS Inspection' '311 Contact Us' 'WATR' 'WRS' 'After Hours Transit'
 'Compliance' 'Opinions on Business Units' 'HR' 'CFD' 'CSC'
 'Animal / Bylaw' 'CBS' 'Partnerships' 'Customer Service & Communications'
 'UEP' 'CAI' 'Law' 'GFL' 'TP' 'CBS Concern' 'Calgary Housing' 'RSP' 'DB

24176

## Agency analysis

In [84]:
agency_nunique_values=df['agency_responsible'].nunique()
print(agency_nunique_values)


agency_unique_values=df['agency_responsible'].unique()
print(agency_unique_values)

agency_unique_values_cnt=df['agency_responsible'].value_counts()
print(agency_unique_values_cnt)

df['new_agencyResponsible'] = df['agency_responsible'].str.split(' -').str[0] 
ag_unique_values = df['new_agencyResponsible'].unique()
print(ag_unique_values)


ag_unique_values = df['new_agencyResponsible'].nunique()
print("Number of Unique values: ",ag_unique_values)
ag_unique_values_cnt = df['new_agencyResponsible'].value_counts()
print(ag_unique_values_cnt)


77
['CFOD - Finance' 'CS - Recreation and Social Programs'
 'CS - Calgary Neighbourhoods' 'OS - Calgary Transit'
 'CS - Calgary Recreation' 'TRAN - Calgary Transit' 'CS - Calgary Parks'
 'CS - Calgary Community Standards' 'UEP - Water Services'
 'OS - Parks and Open Spaces' 'TRAN - Roads'
 'PD - Calgary Building Services'
 'CFOD - Customer Services and Communications' 'UEP - Water Resources'
 'UEP - Waste and Recycling Services' 'CPFS - Assessment and Tax'
 'Corporate Wide Service Requests'
 'CS - Emergency Management and Community Safety'
 'OS - Waste and Recycling Services' 'PICS - Human Resources'
 'CS - Calgary Fire' 'OS - Mobility' 'Tranc - Calgary Transit'
 'Partnerships' 'DCMO - Corporate Analytics and Innovation' 'LL - Law'
 'Uepc - Waste and Recycling Services' 'TRAN - Transportation Planning'
 'OS - Water Services' 'CS - Calgary Housing'
 'Recreation and Social Programs'
 'PDS - Development, Business and Building Services'
 'DCMO - Facility Management' 'Elected Officials'
 'P